In [1]:
import os
import keras
import numpy as np
import pandas as pd
import zipfile
import keras.backend as K

from time import time

from keras import callbacks
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Dense, Input
from keras.initializers import VarianceScaling
from keras.engine.topology import Layer, InputSpec

from scipy.misc import imread
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, normalized_mutual_info_score

import boto3
from io import BytesIO
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ops').getOrCreate()
from pyspark.ml.clustering import KMeans as KMeansSpark
from pyspark.ml.feature import VectorAssembler

Using TensorFlow backend.


In [2]:
'''
-----------------------------------------------------------------------------
--- Inputs
-----------------------------------------------------------------------------
'''
Interp = 'Interp'
dataSource = 'L6'
Coor = 'Polar'
dataSetName = '{}polarInterpDataScaled.csv'.format(dataSource)
DRrange = range(4,16,1)
clusterRange1 = range(2,101,1)
clusterRange2 = range(105,501,5)

# Initialise settings
seed = 128
init = 'glorot_uniform'
pretrain_optimizer = 'adam'
batch_size = 180
maxiter = 2e4
tol = 0.001
update_interval = 50
pretrain_epochs = 1000

In [3]:
'''
-----------------------------------------------------------------------------
--- Definitions
-----------------------------------------------------------------------------
'''
Results = 'Results'
Indicies = 'Indicies'
Response = 'Response'
Figures = 'Figures'

# Configuring S3
s3_bucket_name = 'jasper-ml-sagemaker'
role = get_execution_role()

client = boto3.client('s3')
resource = boto3.resource('s3')
my_bucket = resource.Bucket(s3_bucket_name)

In [4]:
'''
-----------------------------------------------------------------------------
--- Building AutoEncoder & Encoder
-----------------------------------------------------------------------------
'''
def autoencoder(dims, act='relu', init='glorot_uniform'):

    n_stacks = len(dims) - 1
    # input
    x = Input(shape=(dims[0],), name='input')
    h = x

    # internal layers in encoder
    for i in range(n_stacks-1):
        h = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(h)

    # hidden layer
    h = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(h)  # hidden layer, features are extracted from here

    y = h
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        y = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(y)

    # output
    y = Dense(dims[0], kernel_initializer=init, name='decoder_0')(y)

    return Model(inputs=x, outputs=y, name='AE'), Model(inputs=x, outputs=h, name='encoder')

'''
-----------------------------------------------------------------------------
--- Building Clustering Layer
-----------------------------------------------------------------------------
'''

class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape = (self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
                 q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

'''
-----------------------------------------------------------------------------
--- Building Deep Embedded Clustering Class
-----------------------------------------------------------------------------
'''
    
class DEC(object):
    def __init__(self,
                 dims,
                 autoencoder,
                 encoder,
                 n_clusters=10,
                 alpha=1.0,
                 init='glorot_uniform'):

        super(DEC, self).__init__()

        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1

        self.n_clusters = n_clusters
        self.alpha = alpha
        self.autoencoder = autoencoder
        self.encoder= encoder
        
        # prepare DEC model
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')(self.encoder.output)
        self.model = Model(inputs=self.encoder.input, outputs=clustering_layer)

    def load_weights(self, weights):  # load weights of DEC model
        self.model.load_weights(weights)

    def extract_features(self, x):
        return self.encoder.predict(x)

    def predict(self, x):  # predict cluster labels using the output of clustering layer
        q = self.model.predict(x, verbose=0)
        return q.argmax(1)

    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def compile(self, optimizer='sgd', loss='kld'):
        self.model.compile(optimizer=optimizer, loss=loss)

    def fit(self, x, y=None, maxiter=2e4, batch_size=256, tol=1e-3,
            update_interval=140, save_dir='./results/temp'):

        print('Update interval', update_interval)
        save_interval = x.shape[0] / batch_size * 5  # 5 epochs
        print('Save interval', save_interval)

        # Step 1: initialize cluster centers using k-means
        t1 = time()
        print('Initializing cluster centers with k-means.')
        
        E_data = spark.createDataFrame(pd.DataFrame(self.encoder.predict(x)))
        assembler = VectorAssembler(inputCols= E_data.columns,
                                    outputCol = 'features')
        vectorised_E_data = assembler.transform(E_data).select("features")
        kmeansSpark = KMeansSpark(k=self.n_clusters, maxIter= 300,featuresCol='features')
        kMeansModelSpark = kmeansSpark.fit(vectorised_E_data)
        
        y_pred = kMeansModelSpark.transform(vectorised_E_data).select('prediction').toPandas()
        y_pred_last = np.copy(y_pred)
        self.model.get_layer(name='clustering').set_weights([np.array(kMeansModelSpark.clusterCenters())])

        # Step 2: deep clustering
        loss = 0
        index = 0
        index_array = np.arange(x.shape[0])
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)  # update the auxiliary target distribution p

                # evaluate the clustering performance
                y_pred = q.argmax(1)
                if y is not None:
                    acc = np.round(metrics.acc(y, y_pred), 5)
                    nmi = np.round(metrics.nmi(y, y_pred), 5)
                    ari = np.round(metrics.ari(y, y_pred), 5)
                    loss = np.round(loss, 5)
                    logdict = dict(iter=ite, acc=acc, nmi=nmi, ari=ari, loss=loss)
                    logwriter.writerow(logdict)
                    print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)

                # check stop criterion
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = np.copy(y_pred)
                if ite > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    #logfile.close()
                    break

            # train on batch
            if index == 0:
                 np.random.shuffle(index_array)
            idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
            self.model.train_on_batch(x=x[idx], y=p[idx])
            index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

            # save intermediate model
            #if ite % save_interval == 0:
                #print('saving model to:', save_dir + '/DEC_model_' + str(ite) + '.h5')
                #self.model.save_weights(save_dir + '/DEC_model_' + str(ite) + '.h5')

            ite += 1

        # save the trained model
        
        #print('saving model to:', save_dir + '/DEC_model_final.h5')
        #self.model.save_weights(save_dir + '/DEC_model_final.h5')

        return y_pred

In [ ]:
'''
-----------------------------------------------------------------------------
--- Loading Data
-----------------------------------------------------------------------------
'''
dataPath = 'Data/'
dataKey = dataSetName
obj = client.get_object(Bucket=s3_bucket_name, Key=dataPath + dataKey)
dataset = pd.read_csv(obj['Body'],header = None, index_col=None)

'''
-----------------------------------------------------------------------------
--- Performing Analysis
-----------------------------------------------------------------------------
'''
for i in DRrange:
    
    
    Dimensions = i # To be checked
    dims = [dataset.shape[-1],i]
    
    
    # Build encoders
    auto_encoder, encoder = autoencoder(dims, act='relu', init='glorot_uniform')
    auto_encoder.compile(optimizer=pretrain_optimizer, loss='mse')
    t0 = time()
    train_history = auto_encoder.fit(dataset.values, dataset.values, batch_size=batch_size, epochs=pretrain_epochs)
    print('Pretraining time: ', time() - t0)
    
    # Apply Encoder
    pred_auto = encoder.predict(dataset.values).astype(np.float64)
    
    # Prepare data for Spark KMeans
    AE_data = spark.createDataFrame(pd.DataFrame(pred_auto))
    assembler = VectorAssembler(inputCols= AE_data.columns,
                                outputCol = 'features')
    vectorised_AE_data = assembler.transform(AE_data).select("features")
    
    # Define path for saving in s3
    Analysis = 'AE_Simple'
    resultsPath = '{}/{}/{}/{}/{}/{}/'.format(dataSource,Coor,Interp,Analysis,Results,Dimensions)
    collectedIndicies = [] 
    for j in clusterRange1:
        
        #  Perform Kmeans on the data
        timestart = time()
        kmeans = KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
                        n_clusters=j, n_init=20, n_jobs=-1, precompute_distances='auto',
                        random_state=seed, tol=0.0001, verbose=0)
        predictions = kmeans.fit_predict(pred_auto)
        
        # Save and upload to s3
        dataKey = '{}results.csv'.format(j)
        np.savetxt(dataKey,predictions,delimiter=',')
        my_bucket.upload_file(dataKey,Key=resultsPath + dataKey)
        os.remove(dataKey)
        
        # Computing Char Indicies
        profile2 = pd.DataFrame(pred_auto)
        profile2['labels'] = predictions.astype(np.float64)
        charProfile = profile2.groupby(['labels']).mean().values
        characteristicIndicies = []
        for jj in range(len(charProfile)):
            characteristicIndicies.append(np.argmin(np.linalg.norm(pred_auto-charProfile[jj],axis=1)))
        
        collectedIndicies.append(np.flip(characteristicIndicies,axis=0))
        
        print('Straight AE {} elements & {} centres took {} seconds'.format(i,j,time()-timestart))
    
    for j in clusterRange2:
        
        #  Perform Kmeans on the data
        timestart = time()
        kmeansSpark = KMeansSpark(k=j, maxIter= 300,featuresCol='features',seed = seed)
        kMeansModelSpark = kmeansSpark.fit(vectorised_AE_data)
        predictions = kMeansModelSpark.transform(vectorised_AE_data).select('prediction').toPandas()
        
        # save at each stage to reduce memory consumption
        dataKey = '{}results.csv'.format(j)
        predictions.to_csv(dataKey,index=False)
        my_bucket.upload_file(dataKey,Key=resultsPath + dataKey)
        os.remove(dataKey)
        
        # Computing Char Indicies
        profile2 = pd.DataFrame(pred_auto)
        profile2['labels'] = predictions.astype(np.float64)
        charProfile = profile2.groupby(['labels']).mean().values
        characteristicIndicies = []
        for jj in range(len(charProfile)):
            characteristicIndicies.append(np.argmin(np.linalg.norm(pred_auto-charProfile[jj],axis=1)))
        
        collectedIndicies.append(np.flip(characteristicIndicies,axis=0))
        
        print('Straight AE {} elements & {} centres took {} seconds'.format(i,j,time()-timestart))
      
    #Uploading Char Indicies
    indiciesPath = '{}/{}/{}/{}/{}/'.format(dataSource, Coor, Interp, Analysis, Indicies)
    dataKey = '{}DimCharIndicies.npy'.format(Dimensions)
    np.save(dataKey,collectedIndicies)
    my_bucket.upload_file(dataKey,Key=indiciesPath + dataKey)
    os.remove(dataKey)
    
    Analysis = 'DEC_Simple'
    resultsPath = '{}/{}/{}/{}/{}/{}/'.format(dataSource,Coor,Interp,Analysis,Results,Dimensions)
    collectedIndicies = []   
    for j in clusterRange1:
        
        timestart = time()
        dec = DEC(dims=dims,n_clusters=j ,init = init,autoencoder = auto_encoder,encoder = encoder)
        dec.compile(optimizer=SGD(0.01, 0.9), loss='kld')
        
        # Save and upload to s3
        dataKey = '{}results.csv'.format(j)
        predictions = dec.fit(x=dataset.values.astype(np.float64), y=None)
        np.savetxt(dataKey, predictions,delimiter=',')
        my_bucket.upload_file(dataKey,Key=resultsPath + dataKey)
        os.remove(dataKey)
        
        profile1 = pd.DataFrame(dec.model.predict(dataset.values.astype(np.float64)))
        profile2 = profile1.copy()
        profile2['labels'] = predictions
        charProfile = profile2.groupby(['labels']).mean().values
        characteristicIndicies = []
        for jj in range(len(charProfile)):
            characteristicIndicies.append(np.argmin(np.linalg.norm(profile1.values-charProfile[jj],axis=1)))
        
        collectedIndicies.append(np.flip(characteristicIndicies,axis=0))
        
        print('DEC {} elements & {} centres took {} seconds'.format(i,j,time()-timestart))
      
    for j in clusterRange2:
        
        timestart = time()
        dec = DEC(dims=dims,n_clusters=j ,init = init,autoencoder = auto_encoder,encoder = encoder)
        dec.compile(optimizer=SGD(0.01, 0.9), loss='kld')
        
        # Save and upload to s3
        dataKey = '{}results.csv'.format(j)
        predictions = dec.fit(x=dataset.values.astype(np.float64), y=None)
        np.savetxt(dataKey, predictions,delimiter=',')
        my_bucket.upload_file(dataKey,Key=resultsPath + dataKey)
        os.remove(dataKey)
        
        profile1 = pd.DataFrame(dec.model.predict(dataset.values.astype(np.float64)))
        profile2 = profile1.copy()
        profile2['labels'] = predictions
        charProfile = profile2.groupby(['labels']).mean().values
        characteristicIndicies = []
        for jj in range(len(charProfile)):
            characteristicIndicies.append(np.argmin(np.linalg.norm(profile1.values-charProfile[jj],axis=1)))
        collectedIndicies.append(np.flip(characteristicIndicies,axis=0))
        
        print('DEC {} elements & {} centres took {} seconds'.format(i,j,time()-timestart))
    
    
    #Uploading Char Indicies
    indiciesPath = '{}/{}/{}/{}/{}/'.format(dataSource, Coor, Interp, Analysis, Indicies)
    dataKey = '{}DimCharIndicies.npy'.format(Dimensions)
    np.save(dataKey,collectedIndicies)
    my_bucket.upload_file(dataKey,Key=indiciesPath + dataKey)
    os.remove(dataKey)


Epoch 1/1000
11359/11359 [==============================] - 1s 129us/step - loss: 0.7812
Epoch 2/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.3894
Epoch 3/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.3065
Epoch 4/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2807
Epoch 5/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2609
Epoch 6/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2503
Epoch 7/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2461
Epoch 8/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2443
Epoch 9/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2438
Epoch 10/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2436
Epoch 11/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2435
Epoch 12/1000
11359/11359 [=

11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 91/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2434
Epoch 92/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 93/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2433
Epoch 94/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2434
Epoch 95/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2434
Epoch 96/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 97/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 98/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 99/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 100/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2434
Epoch 101/1000
11359/11359 [======

11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 182/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 183/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 184/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2432
Epoch 185/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 186/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 187/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 188/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2434
Epoch 189/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 190/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2433
Epoch 191/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 192/1000
11359/1135

11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 273/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 274/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2432
Epoch 275/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 276/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 277/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2432
Epoch 278/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2432
Epoch 279/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2432
Epoch 280/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2435
Epoch 281/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 282/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2432
Epoch 283/1000
11359/1135

11359/11359 [==============================] - 0s 28us/step - loss: 0.2432
Epoch 364/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 365/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2432
Epoch 366/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 367/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 368/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 369/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 370/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 371/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 372/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2432
Epoch 373/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 374/1000
11359/1135

...

11359/11359 [==============================] - 0s 29us/step - loss: 0.2431
Epoch 910/1000
11359/11359 [==============================] - 0s 29us/step - loss: 0.2434
Epoch 911/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 912/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2431
Epoch 913/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2431
Epoch 914/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2431
Epoch 915/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2431
Epoch 916/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2431
Epoch 917/1000
11359/11359 [==============================] - 0s 27us/step - loss: 0.2432
Epoch 918/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2433
Epoch 919/1000
11359/11359 [==============================] - 0s 28us/step - loss: 0.2431
Epoch 920/1000
11359/1135

11359/11359 [==============================] - 0s 28us/step - loss: 0.2431
Pretraining time:  317.55244421958923
Straight AE 4 elements & 2 centres took 1.867997169494629 seconds
Straight AE 4 elements & 3 centres took 0.24744391441345215 seconds
Straight AE 4 elements & 4 centres took 0.37610292434692383 seconds
Straight AE 4 elements & 5 centres took 0.3494141101837158 seconds
Straight AE 4 elements & 6 centres took 0.5584018230438232 seconds
Straight AE 4 elements & 7 centres took 0.49054741859436035 seconds
Straight AE 4 elements & 8 centres took 0.6201586723327637 seconds
Straight AE 4 elements & 9 centres took 0.5723316669464111 seconds
Straight AE 4 elements & 10 centres took 0.6576657295227051 seconds
Straight AE 4 elements & 11 centres took 0.675262451171875 seconds
Straight AE 4 elements & 12 centres took 0.6879620552062988 seconds
Straight AE 4 elements & 13 centres took 0.8285806179046631 seconds
Straight AE 4 elements & 14 centres took 0.9545636177062988 seconds
Straight A

Straight AE 4 elements & 210 centres took 2.404461622238159 seconds
Straight AE 4 elements & 215 centres took 2.1182305812835693 seconds
Straight AE 4 elements & 220 centres took 2.5475072860717773 seconds
Straight AE 4 elements & 225 centres took 2.795278787612915 seconds
Straight AE 4 elements & 230 centres took 2.1264779567718506 seconds
Straight AE 4 elements & 235 centres took 1.7928597927093506 seconds
Straight AE 4 elements & 240 centres took 2.6603763103485107 seconds
Straight AE 4 elements & 245 centres took 2.0710628032684326 seconds
Straight AE 4 elements & 250 centres took 2.717381477355957 seconds
Straight AE 4 elements & 255 centres took 2.365586519241333 seconds
Straight AE 4 elements & 260 centres took 2.045973539352417 seconds
Straight AE 4 elements & 265 centres took 1.80401611328125 seconds
Straight AE 4 elements & 270 centres took 2.1959564685821533 seconds
Straight AE 4 elements & 275 centres took 1.870962142944336 seconds
Straight AE 4 elements & 280 centres took 

DEC 4 elements & 26 centres took 116.41467618942261 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 4 elements & 27 centres took 117.37622714042664 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 4 elements & 28 centres took 116.42204999923706 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 4 elements & 29 centres took 120.63903164863586 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 4 elements & 30 centres took 122.49643230438232 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 4 elements & 31 centres took 124.3120551109314 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 4 elements & 32 centres took 125.16093444824219 seconds
Update interval 140
Save interval 221.855

DEC 6 elements & 30 centres took 259.2402198314667 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 6 elements & 31 centres took 265.2938377857208 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 6 elements & 32 centres took 264.89225220680237 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 6 elements & 33 centres took 264.0794053077698 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
DEC 6 elements & 34 centres took 267.2639374732971 seconds
Update interval 140
Save interval 221.85546875
Initializing cluster centers with k-means.
